Previous notebooks, and completed processes:

_"preprocessing.ipynb"_
- 1. Initial data adjustments
- 2. Preprocessing

_"baseline scores after preproc.ipynb"_

- Establish baseline scores

# setup

In [ ]:
# script for standard imports
%run '/content/drive/MyDrive/Data_Science/scripts/setup.ipynb'

# read-in data ("trx_preproc" represents the preprocessed trainx data)
trx_preproc = \
pd.read_csv('/content/drive/MyDrive/Data_Science/data/gtd/trx_preproc.csv')

# Step 3: Data transformations 
_(numeric/categoric only)_

3.0.1 : `select_cols_for_pipe`
- Column Selection for Pipelines: creating explicit lists per dtype for sklearn pipeline (next notebook, "transformations")

3.0.2 : Build transformation pipelines
- Impute, encode, scale and removing excessive features that are highly collinear

3.0.3 : Combine numeric/categoric preparation pipelines
- Build/train heterogeneous pipeline on preprocessed data and execute transforms

3.0.1 : `select_cols_for_pipe` - Column Selection for pipelines

In [ ]:
# run script for simple function that uses
# `ColumnSelector` transformer to get columns per dtype
%run '/content/drive/MyDrive/Data_Science/modules/gtd/select_cols_for_pipe.ipynb'

In [ ]:
ncols,ccols,tcols = select_cols_for_pipe(trx_preproc)

# view ColumnSelector objects
display(type(ncols),type(ccols),type(tcols))

mlxtend.feature_selection.column_selector.ColumnSelector

mlxtend.feature_selection.column_selector.ColumnSelector

mlxtend.feature_selection.column_selector.ColumnSelector

We can use the ColumnSelector-transformers objects above, as is, for an individual step of a pipeline. For simplicity, lets just establish the feature-lists from them for now:

In [ ]:
ncols, ccols, tcols = \
list(ncols.cols), list(ccols.cols), list(tcols.cols)

print(f'Total features per dtype:\n\
numeric: {len(ncols)}\ncategoric:{len(ccols)}\ntext:{len(tcols)}')

Total features per dtype:
numeric: 31
categoric:30
text:4


3.0.2 : Build pipelines for standard data transformations, for numeric/categoric training data
- _Impute, encode, scale and removing excessive features that are highly collinear_

In [ ]:
# run script for simple feature-selection 
# function for removing data with high cardinality
%run '/content/drive/MyDrive/Data_Science/modules/gtd/CollinearRemover.ipynb'

In [ ]:
#numeric pipe
nprep_pipe = Pipeline(
    [("n_imp", SimpleImputer(strategy='median')),
    ('n_ssscale', MinMaxScaler()),
    ('n_collin_remove_90', CollinearRemover(collinear_cutoff=.90))])

#categoric pipe
cprep_pipe = Pipeline(
    [('c_imp', SimpleImputer(strategy='constant',fill_value='missing')),
    ('c_hash_enc', ce.HashingEncoder(drop_invariant=True)),
    ('c_collin_remove_95', CollinearRemover(collinear_cutoff=.95))])

3.0.3 : Combine preparation pipelines
- _Build/train heterogeneous pipeline on preprocessed data and execute transforms_

In [ ]:
trx_preproc.shape #reminder

(153171, 65)

In [ ]:
#build heterogeneous prep pipeline 
ncprep_pipe = \
ColumnTransformer([('nprep', nprep_pipe, ncols),
                   ('cprep', cprep_pipe, ccols)])
# train pipeline,
ncprep_pipe.fit(trx_preproc)

# prepare (numeric/categoric) data for ml 
trx_prepped = ncprep_pipe.transform(trx_preproc)

# view results after data transformations (n&c only)
trx_prepped.shape

In [ ]:
# write transformed data (num/cat only), "trx_prepped", to csv
trx_prepped.to_csv('/content/drive/MyDrive/Data_Science/data/gtd/trx_prepped.csv',
                   index=False)

# Next Steps

Hyperparameter tuning and model evaluation (see, _"tuning, scoring, evaluation.ipynb"_).